In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('SN_total.csv')

In [6]:
df.columns

Index(['시간', '원수 탁도', '원수 pH', '원수 알칼리도', '원수 전기전도도', '원수 수온', '3단계 원수 유입 유량',
       '3단계 1계열 응집제 주입률', '3단계 침전지 탁도', '3단계 침전지 체류시간', '3단계 여과지 탁도'],
      dtype='object')

In [9]:
dff = df[['원수 탁도', '원수 pH', '원수 알칼리도', '원수 전기전도도', '원수 수온', '3단계 원수 유입 유량', '3단계 침전지 체류시간', '3단계 1계열 응집제 주입률']]
dff

,원수 탁도,원수 pH,원수 알칼리도,원수 전기전도도,원수 수온,3단계 원수 유입 유량,3단계 침전지 체류시간,3단계 1계열 응집제 주입률
0,2.477549,8.257175,54.961955,222.781944,4.616253,11183.916667,256.817051,13.54400
1,2.439258,8.271153,55.155387,222.880161,4.629616,11205.916667,260.570714,13.55400
2,2.421914,8.282444,55.320712,224.263970,4.629758,11503.791667,270.520977,13.59700
3,2.398210,8.286200,55.872316,224.579963,4.626820,11781.125000,260.697341,13.66800
4,2.408023,8.288734,56.368194,224.477999,4.620842,12237.833333,238.363311,13.68725
...,...,...,...,...,...,...,...,...
51565,1.787780,7.822680,41.094511,184.020745,3.158893,9660.916667,301.454702,12.99750
51566,1.784237,7.820758,40.703410,183.270915,3.163575,9705.458333,293.208228,12.99750
51567,1.765750,7.818412,40.405968,182.062842,3.166926,9854.541667,289.729565,12.99750
51568,1.785479,7.814997,40.534102,182.261220,3.177002,9093.500000,302.390668,12.99750


In [10]:

import streamlit as st
import pandas as pd
import seaborn as sns
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import plotly.graph_objects as go
import matplotlib.pyplot as plt


plt.rcParams['font.family'] = 'Malgun Gothic'  # 또는 출력된 다른 이름
plt.rcParams['axes.unicode_minus'] = False  # 한글 폰트 사용 시, 마이너스 기호 깨짐 방지

@st.cache_data
def load_data():
    # 데이터 불러오기
    df = pd.read_csv('SN_total.csv')

    df["로그 원수 탁도"] = np.log10(df["원수 탁도"])
    df["로그 응집제 주입률"] = np.log10(df["3단계 1계열 응집제 주입률"])
    
    X = df[
        [
            "로그 원수 탁도",
            "원수 pH",
            "원수 알칼리도",
            "원수 전기전도도",
            "원수 수온",
            "3단계 원수 유입 유량",
            "3단계 침전지 체류시간",
            "로그 응집제 주입률"
        ]
    ]
    # Xt, Xts, yt, yts = train_test_split(X, y, test_size=0.2, shuffle=False)

    return X





2025-04-18 15:30:51.066 No runtime found, using MemoryCacheStorageManager


In [12]:
# load data
dff = load_data()

# Select Variable
st.markdown("## Select Target Variable")
column = st.selectbox("Target 변수를 선택하세요.", dff.columns.tolist())
st.dataframe(dff[[column]].head())

st.markdown("## Select Input Variables")
input_columns = st.multiselect("복수의 컬럼을 선택하세요.", dff.columns.tolist())
filtered_col = dff[input_columns]

st.dataframe(dff[input_columns].head())

Xt, Xts, yt, yts = train_test_split(filtered_col, dff[column], test_size=0.2, shuffle=False)



# Hyperparameters
max_depth = st.slider("Select max depth", min_value=0, max_value=20, value=3)
n_estimators = st.slider("n_estimators", min_value=20, max_value=500, value=50)
learning_rate = st.slider("learning_rate", min_value=0.00, max_value=1.00, step=0.01, value=0.1)
subsample = st.slider("subsample", min_value=0.00, max_value=1.00, step=0.01, value=0.8)

xgb = XGBRegressor(
    random_state=2, 
    n_jobs=-1,
    max_depth=max_depth,
    n_estimators=n_estimators,
    learning_rate=learning_rate,
    subsample=subsample
)

xgb.fit(Xt, yt)

yt_pred = xgb.predict(Xt)
yts_pred = xgb.predict(Xts)

mse_train = mean_squared_error(10**yt, 10**yt_pred)
mse_test = mean_squared_error(10**yts, 10**yts_pred)
# st.write(f"학습 데이터 MSE: {mse_train}")
# st.write(f"테스트 데이터 MSE: {mse_test}")

r2_train = r2_score(10**yt, 10**yt_pred)
r2_test = r2_score(10**yts, 10**yts_pred)
# st.write(f"학습 데이터 R2: {r2_train}")
# st.write(f"테스트 데이터 R2: {r2_test}")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ax = axes[0]
ax.scatter(Xt["로그 원수 탁도"], yt, s=3, label="학습 데이터 (실제)")
ax.scatter(Xt["로그 원수 탁도"], yt_pred, s=3, label="학습 데이터 (예측)", c="r")
ax.grid()
ax.legend(fontsize=13)
ax.set_xlabel("로그 원수 탁도")
ax.set_ylabel("로그 응집제 주입률")
ax.set_title(
    rf"학습 데이터  MSE: {round(mse_train, 4)}, $R^2$: {round(r2_train, 2)}",
    fontsize=18,
)

ax = axes[1]
ax.scatter(Xt["로그 원수 탁도"], yt, s=3, label="학습 데이터 (실제)")
ax.scatter(Xt["로그 원수 탁도"], yt_pred, s=3, label="학습 데이터 (예측)", c="r")
ax.grid()
ax.legend(fontsize=13)
ax.set_xlabel("로그 원수 탁도")
ax.set_ylabel("로그 응집제 주입률")
ax.set_title(
    rf"테스트 데이터  MSE: {round(mse_test, 4)}, $R^2$: {round(r2_test, 2)}",
    fontsize=18,
)

# st.pyplot(fig)

2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-18 15:31:18.921 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

IndexError: list index out of range